# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distilbert-base-uncased
* Evaluation approach: I am evaluating classification accuracy by comparing predicted and actual labels.
* Fine-tuning dataset: rkf2778/amazon_reviews_mobile_electronics

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

my_model_name = "distilbert-base-uncased"

id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

# Criteria #1. Load a pretrained HF model
model = AutoModelForSequenceClassification.from_pretrained(
    my_model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

c:\Users\Cody\Desktop\projects\lwft_hf_model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from datasets import load_dataset

# Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(my_model_name)


# Dataset to be used: Amazon reviews of mobile electronics
my_dataset_name = "rkf2778/amazon_reviews_mobile_electronics"
splits = ["test", "train"]

# Criteria #2a Load dataset
datasets = {
    split: dataset
    for split, dataset in zip(splits, load_dataset(my_dataset_name, split=splits))
}

# Reduce the number of data for efficiency
for split in splits:
    datasets[split] = datasets[split].shuffle(seed=50).select(range(500))
    

train_dataset = datasets["train"]
test_dataset = datasets["test"]
    
# Criteria 2b: Preprocess the dataset inlcuding the star_rating_label representing the sentiment
def preprocess_review(batch):
    tokenized = tokenizer(batch["review_body"], padding=True, truncation=True, max_length=512)
    tokenized["labels"] = [label2id[label] for label in batch["star_rating_label"]]
    return tokenized

# Tokenize the datasets
tokenized_train = train_dataset.map(preprocess_review, batched=True)
tokenized_test = test_dataset.map(preprocess_review, batched=True)


In [3]:
import numpy as np
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Accuracy calculation function and Trainer setup is from Udacity lesson 4.14
def calculate_accuracy(predictions):
    logits, labels = predictions
    predicted_class = np.argmax(logits, axis=1)
    return {"accuracy": (predicted_class == labels).mean()}


# Reference:  Udacity lession 4.14
training_args = TrainingArguments(
    output_dir="./data/original_model_sentiments_output",
    per_device_eval_batch_size=8
)

my_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create a trainer (referenced from Udacity lession 4.14)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)

# Criteria #3: Evaluate the pretrained model
trainer.evaluate()


C:\Users\Cody\AppData\Local\Temp\ipykernel_10896\607905029.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.7221406102180481,
 'eval_model_preparation_time': 0.0,
 'eval_accuracy': 0.356,
 'eval_runtime': 10.7939,
 'eval_samples_per_second': 46.322,
 'eval_steps_per_second': 5.837}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [4]:
from peft import get_peft_model, LoraConfig, TaskType


peft_base_model = AutoModelForSequenceClassification.from_pretrained(
    my_model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# Reference: https://huggingface.co/docs/peft/package_reference/lora
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"], # Layers specific to distilbert
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS
)

# Criteria #4: Create a PEFT model
peft_model = get_peft_model(peft_base_model, peft_config)

for name, param in peft_model.named_parameters():
    if "lora" in name:
        param.requires_grad = True  # LoRA parameters should be trainable
    else:
        param.requires_grad = False

peft_model.print_trainable_parameters()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 147,456 || all params: 67,694,596 || trainable%: 0.2178


In [5]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Criteria #5: Train the PEFT model
peft_training_args = TrainingArguments(
    output_dir="./data/peft_model_sentiments_output_2",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)

peft_trainer.train()

# Criteria #6: Save the PEFT model 
peft_model.save_pretrained("my_peft_lora_distilbert2")


c:\Users\Cody\Desktop\projects\lwft_hf_model\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Cody\AppData\Local\Temp\ipykernel_10896\2791392753.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.503583,0.728000
2,No log,0.354992,0.862000
3,No log,0.382819,0.852000


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [6]:
from peft import AutoPeftModelForSequenceClassification
import pandas as pd


# Criteria #7: Load the saved PEFT model (https://huggingface.co/docs/peft/package_reference/auto_class)
saved_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("my_peft_lora_distilbert2")

# Recreate the PEFT trainer from the saved model
tuned_trainer = Trainer(
    model=saved_peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)


# Criteria #8: Compare and evaluate the fine tuned model
print("Original Model accuracy:")
print(pd.DataFrame([trainer.evaluate()]))

print("Fine-tuned Model accuracy:")
print(pd.DataFrame([tuned_trainer.evaluate()]))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Cody\AppData\Local\Temp\ipykernel_10896\2944450660.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  tuned_trainer = Trainer(


Original Model accuracy:
   eval_loss  eval_model_preparation_time  eval_accuracy  eval_runtime  \
0   0.722141                          0.0          0.356       12.3916   

   eval_samples_per_second  eval_steps_per_second  
0                    40.35                  5.084  
Fine-tuned Model accuracy:


   eval_loss  eval_model_preparation_time  eval_accuracy  eval_runtime  \
0   0.354992                       0.0025          0.862       12.8495   

   eval_samples_per_second  eval_steps_per_second  
0                   38.912                  4.903  
